<a href="https://colab.research.google.com/github/vassilyf/ml/blob/main/REC_homework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162992 sha256=eca49975e0254f932ea359661e70bb600e88b387bca9153cc3d75de0941c49b7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [31]:
import os
import re
from google.colab import files
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
uploaded = files.upload()

Saving ml-latest-small.zip to ml-latest-small.zip


In [ ]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
prefix = 'ml-latest-small'
links = pd.read_csv(os.path.join(prefix, 'links.csv'))
movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))
tags = pd.read_csv(os.path.join(prefix, 'tags.csv'))

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [82]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
def normalize(s):
    return re.sub('[^A-Za-z0-9|]+', '', str(s)).replace('|', ' ').lower()

In [86]:
tags_grouped = tags.copy()
tags_grouped['tag'] = tags_grouped[['movieId', 'tag']].groupby('movieId')['tag'].transform(lambda x: ' '.join(map(lambda s: normalize(s), x)))
tags_grouped = tags_grouped[['movieId', 'tag']].drop_duplicates()

movies_with_tags = movies[['movieId']]\
    .merge(tags_grouped[['movieId', 'tag']], on='movieId')

movies_with_tags.head()

,movieId,tag
0,1,pixar pixar fun
1,2,fantasy magicboardgame robinwilliams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


In [87]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [88]:
movies_with_ratings_tags = movies_with_ratings.merge(movies_with_tags, on='movieId').reset_index(drop=True)
movies_with_ratings_tags.dropna(inplace=True)
movies_with_ratings_tags.head()

,movieId,title,genres,userId,rating,timestamp,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,pixar pixar fun
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,pixar pixar fun
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,pixar pixar fun
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,pixar pixar fun
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,pixar pixar fun


Гибридная модель будет состоять из двух шагов. На первом шаге выполним предварительный отбор фильмов используя коллаборативную фильтрацию (с помощью метода скрытых факторов). На втором шаге уточним выбор, используя признаки по содержанию и метод К ближайших соседей.

Обучаем модель для коллаборативной фильтрации.

In [89]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings_tags.userId,
    'iid': movies_with_ratings_tags.title,
    'rating': movies_with_ratings_tags.rating
})

In [90]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [91]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [92]:
algo = SVD(n_factors=20, n_epochs=20, random_state=1)
algo.fit(trainset)

In [93]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8256


0.8256129928912723

In [95]:
# Индексы, по которым просто получить жанр и теги по названию фильма
title_genres = {}
title_tags = {}

for index, row in movies.iterrows():
    title_genres[row.title] = row.genres

for index, row in movies_with_ratings_tags.iterrows():
    title_tags[row.title] = row.tag


In [108]:
# Сначала получаем предварительный и достаточно длинный список фильмов,
# который будем уточнять на следующем шаге
def recomment_step_one(user_id, n):
    user_movies = movies_with_ratings_tags[movies_with_ratings.userId == user_id].title.unique()

    scores = []
    titles = []

    for movie in movies_with_ratings.title.unique():
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=user_id, iid=movie).est)
        titles.append(movie)

    best_indices = np.argsort(scores)[-1 * n:]
    return [titles[i] for i in reversed(best_indices)]


In [109]:
recomment_step_one(3, 10)

<ipython-input-108-b4b182e3cfdf>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_movies = movies_with_ratings_tags[movies_with_ratings.userId == user_id].title.unique()


['Shawshank Redemption, The (1994)',
 'Yojimbo (1961)',
 'Godfather, The (1972)',
 'Philadelphia Story, The (1940)',
 'Godfather: Part II, The (1974)',
 'Rear Window (1954)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
 'Fight Club (1999)',
 'Paths of Glory (1957)',
 "Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)"]

In [110]:
# Уточняем рекомендуемый список на основе близости по жанру к последнему просмотренному пользователем фильму
def recommend_step_two_by_genre(user_id, titles):
    neigh_genres = NearestNeighbors(n_neighbors=5, metric='euclidean')
    tfidf_genres = TfidfVectorizer()

    # жанры фильмов, выбранных на первом шаге
    movie_genres = []
    for title in titles:
      movie_genres.append(title_genres[title])

    X_train_tfidf = tfidf_genres.fit_transform(movie_genres)
    neigh_genres.fit(X_train_tfidf)

    user_movies = movies_with_ratings_tags[movies_with_ratings.userId == user_id].title.unique()
    last_user_movie = user_movies[-1]

    last_movie_genres = normalize(title_genres[last_user_movie])

    last_movie_genres_tfidf = tfidf_genres.transform([last_movie_genres])
    res = neigh_genres.kneighbors(last_movie_genres_tfidf, return_distance=True)
    return pd.Series([titles[i] for i in res[1][0]])


In [111]:
# Уточняем рекомендуемый список на основе близости по тегам к просмотренному фильму с максимальным рейтингом
def recommend_step_two_by_tag(user_id, titles):
    neigh_tags = NearestNeighbors(n_neighbors=5, metric='euclidean')
    tfidf_tags = TfidfVectorizer()

    # теги фильмов, выбранных на первом шаге
    movie_tags = []
    for title in titles:
      movie_tags.append(title_tags[title])

    X_train_tfidf = tfidf_tags.fit_transform(movie_tags)
    neigh_tags.fit(X_train_tfidf)

    user_movies = movies_with_ratings_tags[movies_with_ratings.userId == user_id].sort_values('rating', ascending=False)
    best_movie_tags = user_movies.iloc[0]['tag']

    best_movie_tags_tfidf = tfidf_tags.transform([best_movie_tags])
    res = neigh_tags.kneighbors(best_movie_tags_tfidf, return_distance=True)
    return pd.Series([titles[i] for i in res[1][0]])

In [115]:
# Собственно собранная гибридная система
titles = recomment_step_one(3, 20)
res_by_genre = recommend_step_two_by_genre(3, titles)
res_by_tags = recommend_step_two_by_tag(3, titles)
combined = pd.Series().append(res_by_genre).append(res_by_tags).unique()
combined

<ipython-input-108-b4b182e3cfdf>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_movies = movies_with_ratings_tags[movies_with_ratings.userId == user_id].title.unique()
<ipython-input-110-91ed3ab75846>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_movies = movies_with_ratings_tags[movies_with_ratings.userId == user_id].title.unique()
<ipython-input-115-9c24db78a230>:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  combined = pd.Series().append(res_by_genre).append(res_by_tags).unique()
<ipython-input-115-9c24db78a230>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = pd.Series().append(res_by_genre).append(res_by_tags).unique()
<ipython-input-115-9c24db78a230>:5: FutureWarning: The series.append

array(['Fight Club (1999)', 'Usual Suspects, The (1995)',
       'Godfather, The (1972)', 'Godfather: Part II, The (1974)',
       'Shawshank Redemption, The (1994)',
       'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
       'Rear Window (1954)', 'Paths of Glory (1957)',
       'Philadelphia Story, The (1940)'], dtype=object)